In [1]:
from datascience import *
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action='ignore',category=np.VisibleDeprecationWarning)

# Lecture 11 #

In [3]:
s = [1, 2, 3]

In [4]:
s

[1, 2, 3]

In [5]:
t = [1, 'two', Table()]

In [6]:
t

[1, 'two', ]

### Rows from lists

In [8]:
Table().with_columns('Numbers', np.arange(1, 4))

Numbers
1
2
3


In [9]:
drinks = Table(['Drink', 'Cafe', 'Price'])
drinks

Drink,Cafe,Price


In [10]:
drinks = drinks.with_rows([
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
])
drinks

Drink,Cafe,Price
Milk Tea,Asha,5.5
Espresso,Strada,1.75
Latte,Strada,3.25
Espresso,FSM,2


In [11]:
[
    ['Milk Tea', 'Asha', 5.5],
    ['Espresso', 'Strada',  1.75],
    ['Latte',    'Strada',  3.25],
    ['Espresso', "FSM",   2]
]

[['Milk Tea', 'Asha', 5.5],
 ['Espresso', 'Strada', 1.75],
 ['Latte', 'Strada', 3.25],
 ['Espresso', 'FSM', 2]]

## Review: Group by one column

In [12]:
survey = Table.read_table('welcome_survey_sp22.csv')
survey.show(3)

Year,Extroversion,Number of textees,Hours of sleep,Handedness,Pant leg,Sleep position,Pets,Piercings
Second,5,5,8,Left-handed,Left leg in first,On your left side,Ants,0
Fourth,2,3,7,Right-handed,Left leg in first,On your left side,"Beetles, Centipedes, Scorpions",0
First,3,4,8,Right-handed,Right leg in first,On your back,Bird,0


In [13]:
survey.group("Year")

Year,count
Fifth or above,18
First,577
First year In Master's Degree,1
Fourth,96
"I dont understand this question, I'm a first year but se ...",1
Junior Transfer,1
Not an undergrad,3
Second,419
Third,227


In [16]:
(survey
 .select('Year', 'Hours of sleep')
 .group("Year", np.average)
 .take(1, 7, 8, 3))

Year,Hours of sleep average
First,7.19757
Second,7.1778
Third,6.96696
Fourth,6.92188


In [17]:
survey.group(["Handedness", 'Sleep position']).show()

Handedness,Sleep position,count
Both,On your back,2
Both,On your left side,1
Both,On your right side,3
Both,On your stomach,3
Left-handed,On your back,15
Left-handed,On your left side,30
Left-handed,On your right side,31
Left-handed,On your stomach,17
Right-handed,On your back,299
Right-handed,On your left side,315


In [19]:
(survey
 .select("Handedness", 'Sleep position', "Hours of sleep")
 .group(["Handedness", 'Sleep position'], collect=np.average).show()
)

Handedness,Sleep position,Hours of sleep average
Both,On your back,5
Both,On your left side,6
Both,On your right side,8.16667
Both,On your stomach,8.33333
Left-handed,On your back,7.06667
Left-handed,On your left side,6.96667
Left-handed,On your right side,7.04839
Left-handed,On your stomach,7.5
Right-handed,On your back,7.11538
Right-handed,On your left side,7.11905


## Cross-classification: grouping by two columns

In [21]:
survey.group(['Handedness', 'Sleep position']).show(3)

Handedness,Sleep position,count
Both,On your back,2
Both,On your left side,1
Both,On your right side,3


In [22]:
survey.pivot("Handedness", 'Sleep position').show()

Sleep position,Both,Left-handed,Right-handed
On your back,2,15,299
On your left side,1,30,315
On your right side,3,31,451
On your stomach,3,17,176


In [27]:
survey.pivot("Handedness", 'Sleep position',
             "Hours of sleep", np.average).show()

Sleep position,Both,Left-handed,Right-handed
On your back,5,7.06667,7.11538
On your left side,6,6.96667,7.11905
On your right side,8.16667,7.04839,7.11863
On your stomach,8.33333,7.5,7.25284


## Pivot Tables

In [ ]:
survey.group(['Handedness', 'Sleep position']).show()

## Discussion Questions ##

In [28]:
sky = Table.read_table('skyscrapers.csv')
sky = (sky.with_column('age', 2022 - sky.column('completed'))
          .drop('completed'))
sky.show(3)

name,material,city,height,age
One World Trade Center,mixed/composite,New York City,541.3,8
Willis Tower,steel,Chicago,442.14,48
432 Park Avenue,concrete,New York City,425.5,7


In [38]:
# 1. For each city, what’s the tallest building for each material?

sky_p = sky.pivot('material', 'city', 'height', max)
sky_p.show(2)

city,concrete,mixed/composite,steel
Atlanta,264.25,311.8,169.47
Austin,208.15,0,93.6


In [37]:
# 2. For each city, what’s the height difference between the tallest 
#    steel building and the tallest concrete building?
sky_p.with_column("Difference",
                  sky_p.column("steel") - sky_p.column("concrete"))

city,concrete,mixed/composite,steel,Difference
Atlanta,264.25,311.8,169.47,-94.78
Austin,208.15,0,93.6,-114.55
Baltimore,161.24,0,155.15,-6.09001
Boston,121.92,139,240.79,118.87
Charlotte,265.48,239.7,179.23,-86.25
Chicago,423.22,306.94,442.14,18.92
Cincinnati,125,202.69,175,50
Cleveland,125,288.65,215.8,90.8
Columbus,79.25,0,169.3,90.05
Dallas,176.48,280.72,270.06,93.58


In [45]:
# 3. Generate a table of the names of the oldest buildings for each 
#    material for each city:

def take_the_first_one(a):
    return a.item(0)

(sky.sort('age', descending=True)
    .pivot('material', 'city', 'name', take_the_first_one)).show(2)

city,concrete,mixed/composite,steel
Atlanta,Westin Peachtree Plaza,One Atlantic Center,FlatironCity
Austin,One American Center,,University of Texas Tower


In [43]:
(sky.where('city', 'San Francisco')
    .sort('age', descending=True)
    .column('name')
    .item(0))

'Ferry Building'

## Joins ##

In [46]:
drinks

Drink,Cafe,Price
Milk Tea,Asha,5.5
Espresso,Strada,1.75
Latte,Strada,3.25
Espresso,FSM,2


In [47]:
discounts = Table().with_columns(
    'Coupon % off', make_array(10, 25, 5),
    'Location', make_array('Asha', 'Strada', 'Asha')
)
discounts

Coupon % off,Location
10,Asha
25,Strada
5,Asha


In [48]:
combined = drinks.join('Cafe', discounts, 'Location')
combined

Cafe,Drink,Price,Coupon % off
Asha,Milk Tea,5.5,10
Asha,Milk Tea,5.5,5
Strada,Espresso,1.75,25
Strada,Latte,3.25,25


In [49]:
discounted_frac = 1 - combined.column('Coupon % off') / 100
combined.with_column(
    'Discounted Price', 
    combined.column('Price') * discounted_frac
)

Cafe,Drink,Price,Coupon % off,Discounted Price
Asha,Milk Tea,5.5,10,4.95
Asha,Milk Tea,5.5,5,5.225
Strada,Espresso,1.75,25,1.3125
Strada,Latte,3.25,25,2.4375


In [50]:
drinks.join('Cafe', drinks, 'Cafe')

Cafe,Drink,Price,Drink_2,Price_2
Asha,Milk Tea,5.5,Milk Tea,5.5
FSM,Espresso,2,Espresso,2
Strada,Espresso,1.75,Espresso,1.75
Strada,Espresso,1.75,Latte,3.25
Strada,Latte,3.25,Espresso,1.75
Strada,Latte,3.25,Latte,3.25


In [51]:
drinks

Drink,Cafe,Price
Milk Tea,Asha,5.5
Espresso,Strada,1.75
Latte,Strada,3.25
Espresso,FSM,2
